In [1]:
import os, sys

os.chdir('e:\Downloads (E)\@Books\@TPM\@PDPC\@data_analysis\model_build')
sys.path.append('e:\Downloads (E)\@Books\@TPM\@PDPC\@data_analysis\model_build')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
import matplotlib.ticker as ticker  ## special scaling stuff
import cmasher as cmr
import colorcet as cet
from importlib import reload  ## for reloading packages
import pickle
import OFM_postprocess_scripts as post
from importlib import reload

In [5]:
# retrieve files from saved pickles
model_vers = 'v5.2'
exps_batchname = 'Policies and Indirect'
exps_filename = f'model{model_vers}_exps_{exps_batchname}_'
exps_s = post.read_experiments_from_pickle(f'data_output/{exps_filename}.pickleobject')

# ref_batchname = "Reference"
# exps_r = post.read_experiments_from_pickle(f'data_output/{exps_filename }.pickleobject')
# reload(post)
# extract relevant information from pickled experiment object
meta_s = pd.DataFrame.from_dict(exps_s.results_metadata, orient='index')
meta_s['p_FlightBans'] = meta_s['p_FlightBans'].fillna(value=500).astype(int)
meta_s['p_FlightBans_nom'] = meta_s['p_FlightBans'] - exps_s.c_nominal_ref_date
results_s: pd.DataFrame = post.add_extra_metrics(exps_s)

# meta_r = pd.DataFrame.from_dict(exps_r.results_metadata, orient='index')
# meta_r['p_FlightBans_nom'] = meta_r['p_FlightBans'] - exps_r.c_nominal_ref_date
# results_r: pd.DataFrame = post.add_extra_metrics(exps_r)

# invert the policy inputs to be relative again (instead of machine-focused)
policy_inputs = dict((v, k) for k, v in exps_s.p_flightbans_map.items())  # inversion process
exps_s.results_verbose = exps_s.results_postprocess = None
# exps_r.results_verbose = exps_r.results_postprocess = None


In [6]:
# new one for cumulative
# reload(post)
# batch_cols = ['cum_infected']
# newer system to also include new-infected
batch_cols = {'cum_infected': [10000,],}
              # 'infected_new': [1000, 5000]}
# benchmarks_cum = [10000,] # reverse-find days till metric reaches these value(s)
benchmarks_day = np.arange(1,5,1) * 7 # take cumulative on this day
# batch_df = results_s[results_s.u_Rzero < 2.0 ].copy()
batch_df = results_s
batch_out1 = meta_s.copy()

# set up recording
# table_batch = dict()
# out1_namemap = {'cum_infected': 'Icumul',
#                 'infected_new': 'Inew'}
# table_batch = {'b_cumulative': [],
#             'b_day': [], }
# new_pivs = {}  # only for debug
# New implementation
_out1_namemap = {'cum_infected': 'Icumul',
                'infected_new': 'Inew'}
_metrics_per = ['Dsearch', 'Dvalue'] # what metrics are included per batch
batch_names = dict([(f'{_out1_namemap[batch_key]}_{m}', []) for batch_key in batch_cols.keys() for m in _metrics_per])
## current version should give 4-entry dict for Icumul and Inew, with Dsearch and Dvalue each.

def get_rename(b_label:str,
               b_metric_mode:int) -> str:
    # easy conversion of names for readable output

    return f'{_out1_namemap[b_label]}_{_metrics_per[b_metric_mode]}'

for _batch_label, _batch_metric in batch_cols.items():
    # pivot raw results for efficient sorting
    _pivot = batch_df.pivot(values=_batch_label, index='elapsed_nominal', columns='exp_id')
    # 1st set of outputs for benchmarks-cum (reverse-find)
    for _bench in _batch_metric:
        _find_start = _pivot.apply(np.searchsorted, axis=0, v=_bench, side='left') - exps_s.c_nominal_ref_date
        _statlabel: str = f'{get_rename(_batch_label, 0)}_{_bench}'
        batch_out1[_statlabel] = _find_start
        batch_names[get_rename(_batch_label, 0)].append(_statlabel)

    # 2nd set of outputs for benchmarks-day
    _statlabel: str = get_rename(_batch_label, 1)
    _cols2 = [f'{_statlabel}_{bench}' for bench in benchmarks_day]
    batch_out1[_cols2] = _pivot.loc[benchmarks_day, :].T
    batch_names[get_rename(_batch_label, 1)] = _cols2

In [9]:
batch_to_excel = {}

for _batch_name, _tables in batch_names.items():
    # pivot again for output heatmapping
    df2_piv_p = batch_out1.pivot(columns=['p_FlightBans_nom',],
     index=['u_Rzero', 'u_ImportsIndirect'],
     values=_tables)

    # Original
    # df2_piv_p = batch_out1.pivot(columns='p_FlightBans_nom',
    #  index=['u_ImportsIndirect'],
    #  values=_tables)
    # df2_piv_ref = batch_out1[(batch_out1['p_FlightBans_nom'] == 0) & batch_out1['u_ImportsIndirect'] == 0.].pivot(columns='p_FlightBans_nom',
    #                                                                                                                index=['u_Rzero','u_ImportsFlights', 'u_ImportsIndirect'],
    #                                                                                                                values=tables) #.droplevel(1, axis=1)
    # now need to collect all into nice metrics
    # we have u_Rzero, u_imports, u_Flights
    # r_rel_deltas = df2_piv_p.sub(df2_piv_ref, axis=1, level=0) #.drop(labels=0, level='u_ImportsIndirect', axis=1).astype(int)  # special stuff
    # batch_to_reorder = {f'{_batch_name},cumulative_absolute': df2_piv_p,
    #                   f'{_batch_name}, cumulative_relative': r_rel_deltas}
    # batch_to_reorder =

    # for name, mats in batch_to_reorder.items():
    #     if _batch_name == 'b_cumulative':
    #         batch_to_excel[name] = mats.stack(level=0)\
    #                 .reorder_levels([1,2,0], axis=0)\
    #                 .sort_index(axis=0)
    #     elif _batch_name == 'b_day':
    # batch_to_excel[_batch_name] = df2_piv_p\
    #     .reorder_levels([1,0,2], axis=0)\
    #     .sort_index(axis=0)
        # .droplevel(level=0, axis=1)\
    batch_to_excel[_batch_name] = df2_piv_p.droplevel(level=0, axis=1)\


            # mats.unstack(level='u_ImportsFlights')\
            # .stack(level=0)\
            # .reorder_levels(['u_ImportsFlights', 'u_ImportsIndirect'], axis=1)\
            # .sort_index(axis=1)\
            # .reorder_levels([1, 0], axis=0)\
            # .sort_index(axis=0, level=0)

In [10]:

with open(f'output_figures/{model_vers}/{model_vers}_Outputs{exps_filename}.tsv', 'w') as writefile:
    for batch, stats in batch_to_excel.items():
        writefile.write(f"{batch}\n") # write batch name
        stats.to_csv(writefile, sep='\t', mode='a', lineterminator='\n')  # append only
        writefile.write('\n\n\n')  # make some space

# other stuff

In [ ]:

results_sectioned_melt = pd.melt(results_sectioned,
                                 id_vars=['u_ImportsIndirect', 'elapsed_nominal'],
                                 value_vars=['imported_direct',
                                             'imported_indirect', ],
                                 # 'infected_new'],
                                 )

In [ ]:
sns.scatterplot(results_sectioned_melt,
                x='elapsed_nominal',
                y='value',
                hue='variable',
                style='u_ImportsIndirect')

In [ ]:
# look at imported values and their added case contributions
# cumulative imports and cases (different scales though)
# new added cases per compartment

for metric in []:
    pass


In [ ]:
# thinking how to plot the different stuff

# could do cumulative per scenario?

# look at added infections per scenario
added_i_direct = results_s.pivot(values='direct', index='elapsed_nominal', columns='exp_id')
meta_s['add_i_direct'] = added_i_direct.sum(axis=0)
i_direct_cum = added_i_direct.cumsum(axis=0)

added_i_indirect = results_s.pivot(values='indirect', index='elapsed_nominal', columns='exp_id')
meta_s['add_i_indirect'] = added_i_indirect.sum(axis=0)
i_indirect_cum = added_i_indirect.cumsum(axis=0)
# uncertain: sensitivity?
# cumulative number of imports per group (direct, indirect, 2ndary infections)


In [ ]:
cum_total = results_s.pivot(values='cum_infected', index='elapsed_nominal', columns='exp_id')

# batch handling for tables: think about making into function form

In [ ]:
# save batch df process modes

batch_metrics = {
    ('cum_infected', 50000): ('start_50000',),
    ('infected_new', 5000): ('start_5000', 'duration_5000', 'max', 'max_day'),
}

batch_pivs_dict = {}
batch_2ndary_dict = {}

for metrics, metrics2 in batch_metrics.items():

    metric, threshold = metrics

    # metric = 'infected_new'
    # threshold = 5000 # 50k for cumulative, 5k for duration
    piv_p = results_s.pivot(values=metric, index='elapsed_nominal', columns='exp_id')
    df_piv_p = post.add_threshold_time_search(df_to_search=piv_p,
                                              df_to_add=meta_s,
                                              threshold=threshold)
    df_piv_p['u_ImportsInitial'] = df_piv_p['u_ImportsFlights'] + df_piv_p['u_ImportsIndirect']
    # set up reference case
    piv_r = results_r._pivot(values=metric, index='elapsed_nominal', columns='exp_id')
    df_piv_r = post.add_threshold_time_search(df_to_search=piv_r,
                                              df_to_add=meta_r,
                                              threshold=threshold)
    # ensuring equal comparison with the same initial import rate
    # note hardcode at '-30' term
    df_piv_r['u_ImportsInitial'] = df_piv_r['u_ImportsFlights'].copy()
    # collect specific values
    # values = f'start_{threshold}'  # for cumulative 50k
    # values = f'duration_{threshold}'
    batch_pivs_dict[metric] = {'pivot_p': piv_p,
                               'pivot_r': piv_r,
                               'df_piv_p': df_piv_p,
                               'df_piv_r': df_piv_r
                               }

    # conduct 2ndary pivot to convert into matrix form
    for metric2 in metrics2:
        df2_piv_r = df_piv_r._pivot(columns='p_FlightBans_nom',
                                    index=['u_Rzero', 'u_ImportsInitial'],
                                    values=metric2)

        df2_piv_p = df_piv_p._pivot(columns='p_FlightBans_nom',
                                    index=['u_Rzero', 'u_ImportsInitial', 'u_ImportsFlights', 'u_ImportsIndirect'],
                                    values=metric2)
        df2_piv_p = df2_piv_p.drop(labels=1.1, level=0, errors='ignore')  # Custom addition

        # get diff between
        pr_deltas = df2_piv_p.sub(df2_piv_r[0], axis=0).dropna().drop(labels=0, level='u_ImportsIndirect').astype(int)

        r_rel_deltas = df2_piv_p.diff(axis=1, periods=-1)

        batch_2ndary_dict[(metric, metric2)] = {
            'df2_piv_p': df2_piv_p,
            'pr_deltas': pr_deltas,
            'r_rel_deltas': r_rel_deltas,
        }
    #     break
    # break

In [ ]:
# output for excel
# write into file?
with open('output_figures/exps_post_Policies_small indirect_output.tsv', 'w') as writefile:
    for batch, stats in batch_2ndary_dict.items():
        for table_name, table in stats.items():
            if 'max' in batch:  # special
                table = table * 0.2 / 100  # hospitalisation
            writefile.write(f"{batch},{table_name}\n")
            table.to_csv(writefile, sep='\t', mode='a', lineterminator='\n')  # append only
            writefile.write('\n\n\n')  # make some space


# making graphs for importation

In [ ]:
# plot the importation scales on graph
imports_nonflights = exps_s.i_nonflights_scaled  #.drop(labels=0, axis=1)
imports_nonflights.index = imports_nonflights.index - exps_s.c_nominal_ref_date
sns.lineplot(imports_nonflights)

plt.title('Importation Functions into NL, different scales')
plt.ylabel('Imports per day')
plt.xlabel('Days since reference day (26th November 2021)')

save_name = 'importation function scaled'
plt.savefig(f'output_figures/plot_{exps_batchname}_{save_name}_full_linear.jpg', dpi=300)
plt.xlim((None, 10))
plt.savefig(f'output_figures/plot_{exps_batchname}_{save_name}_zoom_linear.jpg', dpi=300)

plt.yscale('log')
plt.gca().set_yticks(ticks=(0.1, 0.5, 1., 5, 10, 50, 100, 500, 1000))
# plt.gca().tick_params(axis='y', which='minor',bottom=True)
ticks_y = ticker.FuncFormatter(lambda x, pos: f'{float(x)}')
plt.gca().yaxis.set_major_formatter(ticks_y)
plt.ylabel('Imports per day (log)')
plt.savefig(f'output_figures/plot_{exps_batchname}_{save_name}_zoom_log.jpg', dpi=300)

# Make graphs for presentation

In [ ]:
#

In [ ]:
dd_df2 = df_piv_p.groupby(by='u_Rzero', group_keys=True)
temp_dict = {}
temp_dict_rel = {}
# get difference vs next import state
# we know that the t_range stays almost constant

metric = 'start_50000'
for i, cat in enumerate(dd_df2):
    name, group = cat

    # get relative difference from nominal date
    t_nom = group[group['p_FlightBans_nom'] == 0][metric].item()
    # get subgroup:
    t_weeks = group[group['p_FlightBans_nom'] % 7 == 0]
    td_nom = t_weeks[metric] - t_nom
    t_early = t_weeks[metric].max()
    t_late = t_weeks[metric].min()
    td_early = t_early - t_nom
    td_late = t_late - t_nom
    td_range = t_early - t_late

    # get relative policy gain
    td_rel = t_weeks.set_index('p_FlightBans_nom')[metric].diff(periods=-1)
    td_rel_range = td_rel.max() - td_rel.min()
    td_rel_avg = td_rel.mean()
    td_rel_med = td_rel.median()

    # write to dict
    temp_dict[i] = {
        'r0': name,
        't_early': t_early,
        't_nominal': t_nom,
        't_late': t_late,
        'td_early': td_early,
        'td_late': td_late,
        'td_range': td_range,
        'td_rel_range': td_rel_range,
        'td_rel_avg': td_rel_avg,
        'td_rel_med': td_rel_med
    }
    temp_dict_rel[name] = td_rel
    break

# dd_df2_stats = pd.DataFrame.from_dict(temp_dict, orient='index')
# dd_td_stats = pd.concat(temp_dict_rel, axis=1)

# dd_df3 = dd_df2_stats.sort_values(by=['u_Rzero','p_FlightBans_nom'])
# dd_df3 = dd_df2_stats[dd_df2_stats['import'].isin(ddelta_scenarios)].sort_values(by=['import','r0'])


# new stuff

In [ ]:
# collect indices of importance
# we do single-variate analysis first, with 0-day case?
d0_sens = results_s.loc[results_s.p_FlightBans == policy_inputs[0], :]
# d0_sens['infected_cum_pct'] = d0_sens['cum_infected'] / exps_s.c_pop_total  # TODO move this to function form

In [ ]:

# we now isolate only the metric (cumulative infections) and relevant states: day, exp_id, policy/uncertainty
# trim out, mostly for readability
# d0_sens = d0_sens.loc[:, ['exp_id','elapsed_policy','imported','cum_infected', 'infected_total','infected_new']]
# get u_REff from meta
d0_meta = meta_s.loc[meta_s['p_FlightBans'] == policy_inputs[0], ['u_REff', 'u_NominalImports']]

d0_p_imported = d0_sens._pivot(values='imported', index='elapsed_policy', columns='exp_id')
d0_p_inf_cumulative = d0_sens._pivot(values='cum_infected', index='elapsed_policy', columns='exp_id')
d0_p_inf_current = d0_sens._pivot(values='infected_total', index='elapsed_policy', columns='exp_id')
# NB: seaborn prefers long data form for plotting!
# TODO: verify if the u_NominalImports scaling is correct!
# 50k-50k.. how to find?
# top-down search and bottom-up search?
threshold = 5000
d0_m_inf_current = post.add_threshold_time_search(df_to_search=d0_p_inf_current,
                                                  df_to_add=d0_meta,
                                                  threshold=threshold)

# make another m df for cumulative

In [ ]:

# d0_mat_duration = d0_m_inf_current.pivot(values=f'duration_{threshold}', index='u_REff', columns='u_NominalImports')
# d0_mat_max = d0_m_inf_current.pivot(values='max', index='u_REff', columns='u_NominalImports')
# d0_mat_maxday = d0_m_inf_current.pivot(values='max_day', index='u_REff', columns='u_NominalImports')
# prolly can plot the max day and duration as a quadrant plot?

# Sensitivity: colour definition

In [ ]:
# get categories from the experiment object
s_R_categories = exps_s.PARAMS_RAW['u_REff']
s_R_highlight = [1.1, 1.5, 2.0, 2.5, 3.0]  # hues to highlight
s_R_dull_colour = 'silver'  # hues to keep in background
s_R_ref_colour = {1.3: 'limegreen'}  # specific highlight for reference line (R=1.3)
s_R_highlight_all = sorted(s_R_highlight + [1.3])

# create custom palette for plotting
palette_s = cmr.get_sub_cmap('cmr.guppy_r', start=0., stop=1., N=len(s_R_highlight))  # make colourful palette
palette_s2 = dict((R, s_R_dull_colour) for R in s_R_categories)  # create all R-value categories in dull colours
palette_s2.update(
    dict(zip(s_R_highlight, palette_s.colors)))  #  change the highlighted categories to have bright colours
palette_s2.update(s_R_ref_colour)  # change the colour of the reference line

# Plot : sensitivity analysis, R on total infectious people

In [ ]:
# let's try that map plot
# plot start and end, with central point as max day
# TODO: be careful that some inputs are floats and are not set to categorical form
# TODO: convert scaling to percentage of population?
plot_data = d0_m_inf_current.loc[(d0_m_inf_current['u_REff'] > 1.001) & (d0_m_inf_current['u_NominalImports'] == 15), :]
# ad hoc additions
plot_data = pd.melt(plot_data, id_vars=['u_REff', 'u_NominalImports', 'max_day', 'max'],
                    value_vars=['start_5000', 'end_5000'],
                    value_name='x')
plot_data['max'] = plot_data['max'] / exps_s.c_pop_total
# palette = cmr.get_sub_cmap('cmr.guppy_r', start=0.1, stop=0.9)
fig1, ax1 = plt.subplots(figsize=(6, 6))
plot_data = plot_data.rename(columns={'u_REff': 'R number',
                                      'u_NominalImports': 'Total Imports'},
                             )
plot_duration = sns.lineplot(plot_data, x='x', y='max', ax=ax1, palette=palette_s2,
                             hue='R number',  # style='Total Imports', dashes=dash_styles,
                             linewidth=2.)
plot_peaks = sns.scatterplot(plot_data, x='max_day', y='max', ax=ax1, palette=palette_s2, hue='R number', legend=False)
# rename axis
ax1.set_ylim(0, 1)
ticks_y = ticker.FuncFormatter(lambda x, pos: f'{int(100 * x)}%')  # black magic
ax1.yaxis.set_major_formatter(ticks_y)
# ticks_x = ticker.FuncFormatter(lambda x, pos: f'{x/28}')
# ax1.xaxis.set_major_formatter(ticks_x)
ax1.set_xticks(ticks=range(0, 501, 30))

ax1.set_ylabel('Peak infectious persons [% of total population]')
ax1.set_xlabel('Days since start of entry restriction (26th November 2021)')
ax1.set_title('Effect of R on outbreak period (line)\n '
              'and peak infectious persons (dot)\n'
              '(R = 1.3 as reference case)')

# get legend handles
h, l = ax1.get_legend_handles_labels()
h = [h[k] for k, a in enumerate(l) if float(a) in s_R_highlight_all]
ax1.legend(handles=h, labels=s_R_highlight_all, title='R number')

plt.tight_layout()
fig1.savefig(f'output_figures/plot_sens_R_peak_duration_{model_vers}.jpg', dpi=300)
# source: https://stackoverflow.com/questions/10171618/changing-plot-scale-by-a-factor-in-matplotlib

# Sensitivity: plot truncated wave plot for cumulative infected people

In [ ]:
plot_data = d0_sens.loc[(d0_sens.u_NominalImports == 15) & (d0_sens.u_REff > 1.), :]
renamer = {'u_REff': 'R number',
           'cum_infected_pct': 'Percentage\ninfected',
           'elapsed_nominal': 'Days since reference date (26th Nov 2021)'}
plot_data = plot_data.rename(columns=renamer)

fig, ax4 = plt.subplots(figsize=(5, 6))
sns.lineplot(plot_data, x=renamer['elapsed_nominal'], y=renamer['cum_infected_pct'],
             hue=renamer['u_REff'], ax=ax4, palette=palette_s2)

ax4.set_xlim(None, 190)
ax4.set_xticks(range(-30, 211, 30))
ticks_y = ticker.FuncFormatter(lambda x, pos: f'{int(x * 100)}%')
ax4.yaxis.set_major_formatter(ticks_y)

ax4.set_xlabel('Days since start of entry restriction (26th November 2021)')
ax4.set_ylabel('Percentage of population infected')
ax4.set_title('Effect of R on outbreak trajectory\n'
              '(R = 1.3 as reference case)')
h, l = ax4.get_legend_handles_labels()
h = [h[k] for k, a in enumerate(l) if float(a) in s_R_highlight_all]
ax4.legend(handles=reversed(h), labels=reversed(s_R_highlight_all), title="R Number")

# ax4.legend(handles=reversed(h), labels=reversed(l),
#            title='R number')

plt.tight_layout()
fig.savefig(f'output_figures/plot_sens, R on outbreak trajectory, {model_vers}.jpg', dpi=300)

# Sensitivity: plot for import scaling on outbreak peak timing

In [ ]:
piv_p = results_s.pivot(values='infected_total', index='elapsed_nominal', columns='exp_id')
threshold = 50000  # 50k
df_piv_p = post.add_threshold_time_search(df_to_search=piv_p,
                                          df_to_add=meta_s,
                                          threshold=threshold)


In [ ]:
# plot peak sizes here
plot_data = df_piv_p.loc[df_piv_p['u_REff'] > 1.001, :]
plot_data = plot_data.rename(columns={'u_REff': 'R number',
                                      'u_NominalImports': 'Total Imports'})
plot_data['max'] = plot_data['max'] / exps_s.c_pop_total
scaling = 1
# plot_sizes = {5: scaling,  # in pt
#               10: 2 * scaling,
#               15: 3 * scaling,
#               20: 4 * scaling}

plot_sizes = {imp: scaling * imp for imp in plot_data['Total Imports'].unique()}
fig2, ax2 = plt.subplots(figsize=(6, 6))
plot2_peaks = sns.scatterplot(plot_data, x='max_day', y='max', hue='R number',
                              size='Total Imports', sizes=plot_sizes,
                              ax=ax2, palette=palette_s2)

ticks_y = ticker.FuncFormatter(lambda x, pos: f"{int(x * 100)}%")
ax2.set_ylim(0, 1)
ax2.yaxis.set_major_formatter(ticks_y)
ax2.set_xticks(ticks=(range(0, 511, 30)))
ax2.set_ylabel('Peak infectious persons [% total population]')
ax2.set_xlabel('Days since start of flight restriction')
ax2.set_title('Effect of R number and import scaling\n'
              'on outbreak peak timing and magnitude\n'
              '(total infectious persons)')

handles, labels = ax2.get_legend_handles_labels()
items = [1, 3, 5, 10, 15, 20]  # indices. hardcoded.
handles_plt, labels_plt = zip(*[(handles[idx], labels[idx]) for idx in items])
# labels_plt = [labels[idx] for idx in [0,2]]
ax2.legend(handles=handles_plt, labels=labels_plt, title='R_number')

plt.tight_layout()
fig2.savefig(f'output_figures/plot_sens_peak_timing, {model_vers}.jpg', dpi=300)
del fig2, ax2

In [ ]:

# specific plot for nominal case
plot_ref = 1.3
plot_data = df_piv_p.loc[df_piv_p['u_REff'] == plot_ref, :].copy()
plot_data['max'] = plot_data['max'] / exps_s.c_pop_total
rn = {'p_FlightBans': 'Entry Restriction Policy [day]',
      'u_NominalImports': 'Total Imports'}
plot_data = plot_data.rename(columns=rn)
fig3, ax3 = plt.subplots(figsize=(5, 5))
palette = 'cet_CET_R3'
# 'blueviolet' reference
palette_2 = dict((k, 'limegreen') for k in policy_inputs.values())
f3_line = sns.lineplot(plot_data, x='max_day', y=rn['p_FlightBans'],
                       hue=rn['p_FlightBans'], palette=palette_2, ax=ax3, legend=False, linewidth=3, zorder=0.9)
f3_scatter = sns.scatterplot(plot_data, x='max_day', y=rn['p_FlightBans'], hue=rn['p_FlightBans'],
                             size=rn['u_NominalImports'],
                             palette=palette_2,
                             ax=ax3, edgecolor='white', linewidth=1,
                             legend=True, sizes=plot_sizes)

ticks_y = ticker.FuncFormatter(lambda x, pos: f"{int(x - exps_s.c_nominal_ref_date)}")
ax3.yaxis.set_major_formatter(ticks_y)
ax3.invert_yaxis()
ax3.yaxis.grid(False)

# ax3.set_xticks(range(65, 101, 5))
ax3.set_xlabel('Days from reference date (26th Nov 2021)')

ax3.set_title('Effect of entry restriction timing '
              'on day of infection peak,\n'
              'across different import rates '
              f'(R = {plot_ref})')

handles, labels = ax3.get_legend_handles_labels()
ax3.legend(handles=handles[-6:], labels=labels[-6:], title='Total Imports')

plt.tight_layout()
fig3.savefig(
    f'output_figures/plot_sens, effect of policy on infection peak, zoom nominal, {model_vers}, {plot_ref}.jpg',
    dpi=300)

del ax3, fig3

# Sensitivity: plot: deltas

In [ ]:
# group by R0 and nominal import scaling, trying to get the difference in TDD (time delta difference).
# TDD describes the gain in days you get from having an earlier flight ban
infc2_m_group = df_piv_p.groupby(by=['u_REff', 'u_NominalImports'], axis=0, group_keys=True)
temp_dict = {}
metric = 'start_50000'

# isolate the start time of the outbreak
for i, cat in enumerate(infc2_m_group):
    name, group = cat
    t_early = group[metric].min()
    t_last = group[metric].max()
    t_range = t_last - t_early

    t_day_deltas = group[metric].diff(periods=-1)
    tdd_avg = t_day_deltas.mean()
    tdd_max = t_day_deltas.max()
    tdd_min = t_day_deltas.min()
    tdd_range = tdd_max - tdd_min

    temp_dict[i] = {
        'r0': name[0],
        'import': name[1],
        't_first': t_early,
        't_last': t_last,
        't_range': t_last - t_early,
        'tdd_max': tdd_max,
        'tdd_min': tdd_min,
        'tdd_avg': tdd_avg,
        'tdd_range': tdd_range,

    }

what_df = pd.DataFrame.from_dict(temp_dict, orient='index')
del temp_dict, metric, i, cat, name, group, t_early, t_last, t_range, t_day_deltas, tdd_min, tdd_max, tdd_avg, tdd_range

In [ ]:
# we know that the t_range, tdd_avg, and tdd_range are the same across R0
# the effect of importation is seen to only affect peak timing
# however, the efficacy of the flight bans over longer durations is not visible, due to the short range of flight bans
# how about difference of period between
what_df['r0_plot'] = what_df.r0 + ((what_df['import'] // 15 - 4) / 120)
what_df['category'] = [str(i) for i in list(zip(what_df['r0'], what_df['import']))]

# massage common palette for plotting
wdf_color_diff = list(zip(what_df['r0'], what_df['import'], what_df['category']))
# create redundant palette
wdf_palette = {}
wdf_makeblack = [15, 120]
for r0, imp, cat in wdf_color_diff:
    if imp in wdf_makeblack:
        wdf_palette[cat] = 'dimgray'
    else:
        wdf_palette[cat] = palette_s2[r0]

what_df_melt = what_df.melt(id_vars=['r0_plot', 'r0', 'import', 'category'], value_vars=['t_first', 't_last'])
fig, ax = plt.subplots(figsize=(4, 6))
sns.lineplot(data=what_df_melt, x='value', y='r0_plot', hue='category', palette=wdf_palette, ax=ax, legend=False)

ax.set_yticks(s_R_highlight_all)
ax.set_xticks(range(0, 301, 60))
ax.set_xlabel('Days since nominal')

In [ ]:
# make heatmap (due to large disparity in t_range)
# we isolate for 5 scenarios: 1.1, 1.5, 2.0, 2.5, 3.0
# look at t_range and tdd
# r_range = [1.1, 1.5, 2., 2.5, 3.]
# what_df_specific = what_df[what_df['r0'].isin(r_range)]
# try plotting the TDDs

# create redundant palette
wdf_palette = {}
wdf_makeblack = []
for r0, imp, cat in wdf_color_diff:
    if imp in wdf_makeblack:
        wdf_palette[cat] = 'dimgray'
    else:
        wdf_palette[cat] = palette_s2[r0]


In [ ]:
# clean up data for histogram/heatmap
# we choose [15, 60, 120]
# try to reference from nominal day
# we want

ddelta_scenarios = [15, 60, 120]
ddelta_df = df_piv_p[df_piv_p['u_NominalImports'].isin(ddelta_scenarios)].drop(
    columns=['p_FlightBans', 'u_TIncub', 'c_SimTime', 'c_PopTotal', 'c_model_engine'])
ddelta_df['t_start_to_max'] = ddelta_df['max_day'] - ddelta_df['start_50000']


In [ ]:
# get relative day difference and day delta from nominal?
# separate according to scenario

dd_df2 = df_piv_p.groupby(by=['u_REff', 'u_NominalImports'], group_keys=True)
temp_dict = {}
temp_dict_rel = {}
# get difference vs next import state
# we know that the t_range stays almost constant

metric = 'start_50000'
for i, cat in enumerate(dd_df2):
    name, group = cat
    # get relative difference from nominal date
    t_early = group[metric].max()
    t_late = group[metric].min()
    t_nom = group[group['p_FlightBans_nom'] == 0][metric].item()
    td_early = t_early - t_nom
    td_late = t_late - t_nom
    td_range = t_early - t_late

    # get relative policy gain
    td_rel = group.set_index('p_FlightBans_nom')[metric].diff(periods=-1)
    td_rel_range = td_rel.max() - td_rel.min()
    td_rel_avg = td_rel.mean()
    td_rel_med = td_rel.median()

    temp_dict[i] = {
        'r0': name[0],
        'import': name[1],
        't_early': t_early,
        't_nominal': t_nom,
        't_late': t_late,
        'td_early': td_early,
        'td_late': td_late,
        'td_range': td_range,
        'td_rel_range': td_rel_range,
        'td_rel_avg': td_rel_avg,
        'td_rel_med': td_rel_med
    }
    temp_dict_rel[name] = td_rel

dd_df2_stats = pd.DataFrame.from_dict(temp_dict, orient='index')
dd_td_stats = pd.concat(temp_dict_rel, axis=1)

# fig, ax = plt.subplots(figsize=(6,6))
# sns.scatterplot(what_df, x='tdd_avg',
#                 y='r0',
#                 hue='category',
#                 palette=wdf_palette,
#                 legend=False,
#                 linewidths=0.,
#                 alpha=0.5,
#                 ax=ax)
# ax.set_yticks(s_R_highlight_all)
# ax.set_xticks(range())


In [ ]:
# first isolate for t_nominal
# what do I want to show? -> how lower R values lead to more effective flight bans
# non-linear relationship
# get dfs and save into excels?
dd_df3 = dd_df2_stats[dd_df2_stats['import'].isin(ddelta_scenarios)].sort_values(by=['import', 'r0'])


In [ ]:
dd_df3.to_csv('output_figures/dd_df_scenarios.csv', sep=',', index=False)